<a href="https://colab.research.google.com/github/OWNA/Liberal/blob/main/GITfresh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Setup Environment (ULTRA CLEAN INSTALL ATTEMPT)

# --- Step 1: Aggressively Uninstall Key Libraries ---
print("Aggressively uninstalling numpy, scipy, pandas, lightgbm, EMD-signal, pandas-ta, shap, scikit-learn...")
!pip uninstall numpy -y --quiet
!pip uninstall scipy -y --quiet
!pip uninstall pandas -y --quiet
!pip uninstall lightgbm -y --quiet
!pip uninstall EMD-signal -y --quiet
!pip uninstall pandas-ta -y --quiet
!pip uninstall shap -y --quiet
!pip uninstall scikit-learn -y --quiet

# --- Step 2: Install NumPy pinned to <2.0 FIRST ---
print("\nInstalling NumPy <2.0 (target 1.26.4)...")
!pip install "numpy==1.26.4" --quiet # Force specific 1.x version

# --- Step 3: Install specific compatible versions of Pandas and SciPy ---
print("\nInstalling specific compatible versions of Pandas and SciPy...")
!pip install "pandas==2.0.3" --quiet # Known to work well with NumPy 1.26.x
!pip install "scipy==1.11.4" --quiet  # Known to work well with NumPy 1.26.x & Pandas 2.0.x

# --- Step 4: Install LightGBM and scikit-learn ---
print("\nInstalling LightGBM and scikit-learn...")
!pip install lightgbm --quiet
!pip install scikit-learn --quiet


# --- Step 5: Install the rest of the libraries (EMD-signal pinned) ---
print("\nInstalling remaining libraries (EMD-signal pinned)...")
!pip install ccxt optuna shap "EMD-signal<1.4.0" matplotlib pyyaml websocket-client dill==0.3.7 pandas_ta --quiet

# --- Step 6: Mount Google Drive ---
print("\nMounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# --- Step 7: Define Base Directory for the Project ---
BOT_BASE_DIR = '/content/drive/MyDrive/trading_bot_project_v2/' # !!! YOUR PATH HERE !!!
import os
os.makedirs(BOT_BASE_DIR, exist_ok=True)
os.environ['BOT_BASE_DIR'] = BOT_BASE_DIR
print(f"BOT_BASE_DIR set to: {BOT_BASE_DIR}")
import sys
if BOT_BASE_DIR not in sys.path:
    sys.path.append(BOT_BASE_DIR)
    print(f"Added {BOT_BASE_DIR} to sys.path for module imports.")

# --- Step 8: Check Python and Library Versions ---
print(f"\n--- Library Versions ---")
print(f"Python: {sys.version.split()[0]}")
try:
    import importlib.metadata

    libs_to_check = {
        "ccxt": "CCXT", "lightgbm": "LightGBM", "pandas": "Pandas",
        "numpy": "NumPy", "optuna": "Optuna", "shap": "SHAP",
        "EMD-signal": "EMD-signal", "matplotlib": "Matplotlib", "scipy": "SciPy",
        "PyYAML": "PyYAML", "websocket-client": "websocket-client", "dill": "Dill",
        "scikit-learn": "scikit-learn", "pandas_ta": "pandas_ta"
    }
    for lib_pkg_name, display_name in libs_to_check.items():
        try:
            version = importlib.metadata.version(lib_pkg_name)
            print(f"{display_name}: {version}")
        except importlib.metadata.PackageNotFoundError:
            print(f"{display_name}: Not installed or version not found.")
        except Exception as e_ver:
            print(f"Error getting version for {display_name}: {e_ver}")
except ImportError:
    print("Could not import 'importlib.metadata'. Manual version checks might be needed.")
except Exception as e_outer:
    print(f"An error occurred during library version checking: {e_outer}")
print("------------------------")

print("\nEnvironment setup complete (ULTRA CLEAN Install Attempt).")
print("IMPORTANT: Ensure all your custom .py files are in the BOT_BASE_DIR and contain plain Python code.")

Aggressively uninstalling numpy, scipy, pandas, lightgbm, EMD-signal, pandas-ta, shap, scikit-learn...

Installing NumPy <2.0 (target 1.26.4)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 110.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.28.0 requires pandas>=1.1.4, which is not installed.
sklearn-pandas 2.2.0 requires pandas>=1.1.4, which is not installed.
sklearn-pandas 2.2.0 requires scikit-learn>=0.23.0, which is not installed.
sklearn-pandas 2.2.0 requires scipy>=1.5.1, which is not installed.
xarray 2025.1.2 requires pandas>=2.1, which is not installed.
pytensor 2.27.1 requires scipy<2,>=1, which is not installed.
seaborn 0.13.2 requires pandas>=1.2, which is not installed.
bqplot 0.12.44 requires pandas<3.0.0,>=1.0.0, which is n

In [2]:
# Cell 2: Configuration Settings

import yaml
import os
import numpy as np # For np.nan if you need to represent null for some specific logic

# --- Ensure BOT_BASE_DIR is available (set in Cell 1) ---
BOT_BASE_DIR = os.environ.get('BOT_BASE_DIR')
if BOT_BASE_DIR is None:
    print("CRITICAL ERROR (Cell 2): BOT_BASE_DIR is not set. Please run Cell 1 first.")
    raise EnvironmentError("BOT_BASE_DIR not set. Run Cell 1 to define it.")

# --- Define Configuration Dictionary (Based on User Input) ---
config = {
    # -- Exchange & Symbol --
    'exchange_name': 'bybit',  # Default, will be overridden if in user YAML
    'exchange_testnet': True,
    'symbol': 'BTC/USDT:USDT', # User provided
    'market_type': 'linear',   # Default, assuming linear from symbol format
    'timeframe': '1m', # User provided

    # -- Data Fetching & Paths --
    'base_dir': BOT_BASE_DIR,  # Uses the BOT_BASE_DIR from environment
    'fetch_ohlcv_limit': 2500, # User provided
    'load_existing_ohlcv': True,
    'l2_data_folder': 'l2_data',
    'l2_log_file': 'l2_data_collector.log',
    'fetch_ohlcv_limit_for_scaling': 750,
    'fetch_ohlcv_limit_wfo': 5000, # User provided

    # -- L2 Data Collector Specific --
    'collector_symbol': 'BTCUSDT', # User provided
    'collector_duration': 1,       # User provided
    'collector_unit': "minutes",   # User provided
    'collector_depth': 50,         # User provided (for WebSocket subscription)
    'collector_category': "linear",# User provided
    'l2_max_file_size_mb': 20,
    'l2_collection_duration_seconds': 60, # Derived from 1 minute for consistency

    # -- Feature Engineering --
    'feature_window': 24,      # User provided
    'ohlcv_base_features': ["z_close", "z_volume", "z_spread"],
    'ta_features': ['rsi', 'macd', 'bbands', 'atr', 'kama', 'supertrend', 'vwap'], # Added from user's ta_indicator_params

    'ta_indicator_params': { # User provided
        'rsi': {'length': 10, 'scalar': 100},
        'macd': {'fast': 8, 'slow': 21, 'signal': 5},
        'bbands': {'length': 20, 'std': 2.5},
        'atr': {'length': 14}, # Default, can be overridden here
        'kama': {'length': 10, 'fast': 2, 'slow': 30},
        'supertrend': {'length': 7, 'multiplier': 3, 'atr_period': 10},
        'vwap': {},
    },

    'use_hht_features': True,
    'hht_features_imf_bases': ['hht_freq_imf', 'hht_amp_imf'],
    'hht_imf_count': 3,
    'hht_emd_noise_width': 0.05,

    'use_l2_features': True,   # User provided
    'use_l2_features_for_training': True,
    'l2_depth_imbalance_levels': [3, 5, 10, 15, 25], # User provided
    'l2_features': [
        'price_impact_10',
        'bid_curve', 'ask_curve'
        # 'depth_imb_X' features are generated based on l2_depth_imbalance_levels
    ],
    'l2_price_impact_depth_idx': 4,
    'l2_curve_fit_levels': 20,
    'l2_depth': 25, # User provided (likely for REST L2 snapshot depth in DataHandler)

    # -- Label Generation --
    'labeling_method': 'triple_barrier', # User provided

    'label_volatility_window': 20, # User provided (for vol_norm_return)
    'label_clip_quantiles': [0.01, 0.99], # User provided (for vol_norm_return)
    'label_shift': -1,         # User provided (for vol_norm_return)

    'triple_barrier_profit_target_atr_mult': 2.5, # User provided
    'triple_barrier_stop_loss_atr_mult': 1.0,   # User provided
    'triple_barrier_time_horizon_bars': 12,     # User provided
    'triple_barrier_atr_column': 'atr',         # User provided

    # -- Model Training --
    'random_state': 42,
    'test_size': 0.2,          # User provided
    'min_training_samples': 100,
    'train_ensemble': False,
    'lgbm_n_jobs': -1,

    'optuna_trials': 100,       # User provided
    'optuna_n_estimators_max': 1500,
    'optuna_early_stopping_rounds': 25,
    'optuna_study_name': f"lgbm_opt_BTC-USDT_1m", # Adjusted based on user's symbol/timeframe
    'optuna_load_if_exists': True,
    'optuna_n_jobs': 1,
    'optuna_timeout_seconds': None,

    'optuna_search_spaces': { # User provided
        'n_estimators': {'type': 'int', 'low': 50, 'high': 800, 'step': 25},
        'learning_rate': {'type': 'float', 'low': 0.005, 'high': 0.1, 'log': True},
        'num_leaves': {'type': 'int', 'low': 15, 'high': 100},
        'max_depth': {'type': 'int', 'low': 2, 'high': 8},
        'lambda_l1': {'type': 'float', 'low': 1e-8, 'high': 10.0, 'log': True},
        'lambda_l2': {'type': 'float', 'low': 1e-8, 'high': 10.0, 'log': True},
        'feature_fraction': {'type': 'float', 'low': 0.4, 'high': 0.9},
        'bagging_fraction': {'type': 'float', 'low': 0.4, 'high': 0.9},
        'bagging_freq': {'type': 'int', 'low': 1, 'high': 10},
        'min_child_samples': {'type': 'int', 'low': 3, 'high': 40}
    },

    'ensemble_long_thresh': 0.5,
    'ensemble_short_thresh': -0.5,
    'ensemble_clf_params': {
        'objective': 'multiclass', 'metric': 'multi_logloss',
        'num_class': 3, 'n_estimators': 200,
    },
    'ensemble_reg_params': {
        'objective': 'regression_l1', 'metric': 'mae',
        'n_estimators': 200,
    },

    'enable_feature_selection': False, # User provided
    'feature_selection_method': 'shap', # User provided
    'num_features_to_select': 30,       # User provided

    # -- Model Prediction & Trading Logic --
    # 'backtest_threshold' from user's list is now 'prediction_threshold' for clarity
    'prediction_threshold': 0.2, # User provided as backtest_threshold
    'use_ensemble_for_backtest': False,
    'use_ensemble_for_simulation': False,
    'use_ensemble_for_visualization': False,

    # -- Risk Management --
    'risk_management': {
        'max_drawdown': 0.20,
        'volatility_lookback': 14,
        'position_sizing_mode': 'volatility_target',
        'volatility_target_pct': 0.02,
        'max_equity_risk_pct': 0.05,
        'fixed_fraction_pct': 0.02,
        # User provided stop_loss_pct: null, take_profit_pct: null.
        # These are not directly used by current ATR-based AdvancedRiskManager.
        # Keeping them if user has other plans, or they can be removed if only ATR-based is used.
        'stop_loss_pct': None, # User provided
        'take_profit_pct': None, # User provided
        'sl_atr_multiplier': 1.5,
        'tp_atr_multiplier': 2.5
    },
    'fallback_volatility_pct_for_sizing': 0.02,

    # -- Order Execution --
    'execution': {
        'slippage_model_pct': 0.0005,
        'max_order_book_levels': 20,
        'default_entry_order_type': 'market', # User provided as simulation_entry_order_type
        'default_exit_order_type': 'limit',  # User provided as simulation_exit_order_type
    },

    # -- Backtesting --
    'initial_balance': 10000, # User provided
    'commission_pct': 0.0006,  # User provided
    'leverage': 3,             # User provided
    'fallback_atr_pct_for_backtest': 0.02,

    # -- Walk-Forward Optimization --
    'run_walk_forward_optimization': False, # Default to False, can be overridden
    'walk_forward_train_periods': 730,      # User provided
    'walk_forward_test_periods': 180,       # User provided
    'walk_forward_step_periods': 180,       # User provided
    'walk_forward_initial_warmup': 100,     # User provided
    'walk_forward_retrain_frequency_folds': 1, # User provided
    # fetch_ohlcv_limit_wfo is already under Data Fetching

    # -- Live Simulation --
    'run_simulation_flag': True, # User provided
    'simulation_threshold': 0.2, # User provided
    'fetch_live_limit': 300,   # User provided
    'min_simulation_interval_seconds': 15,
    'simulation_duration_seconds': 300, # User provided

    # -- Visualization --
    'show_plots': True,
    'plot_style': 'seaborn-v0_8-darkgrid',
    'use_shap_for_importance': True, # User provided as use_shap_override
    'shap_max_samples': 1000,
    'plot_figsize_equity': (14, 7),
    'plot_figsize_lgbm': (12, 10),
    'plot_figsize_shap_bar': (12, 10),
    'plot_figsize_shap_dot': (12, 10),
    'plot_figsize_emd': (14, 12),
    'plot_figsize_features': (14, 10),

    # -- Orchestrator --
    'allow_no_exchange_init': False
}

# --- Overwrite specific keys from user's list if they differ from my structured interpretation ---
# This step ensures the user's exact values (from their snippet) are prioritized
# for the keys they explicitly provided.
user_provided_config_snippet = {
    'base_dir': BOT_BASE_DIR, # This must come from the environment
    'symbol': 'BTC/USDT',
    'timeframe': '1m',
    'feature_window': 24,
    'use_l2_features': True,
    # 'l2_depth_levels': 25, # Renamed to l2_depth_imbalance_levels, and l2_depth is separate
    'l2_depth': 25, # For REST L2 snapshots if DataHandler uses it directly
    'fetch_ohlcv_limit': 2500,
    'optuna_trials': 100,
    'test_size': 0.2,
    'backtest_threshold': 0.2, # This is now 'prediction_threshold'
    'initial_balance': 10000,
    'commission_pct': 0.0006,
    'leverage': 3,
    'stop_loss_pct': None, # Kept as user provided
    'take_profit_pct': None, # Kept as user provided
    'run_simulation_flag': True,
    'simulation_threshold': 0.2,
    'fetch_live_limit': 300,
    'simulation_duration_seconds': 300,
    'use_shap_override': True, # This is now 'use_shap_for_importance'
    'collector_symbol': 'BTCUSDT',
    'collector_duration': 1,
    'collector_unit': 'minutes',
    'collector_depth': 50,
    'collector_category': 'linear',
    'labeling_method': 'triple_barrier',
    'label_volatility_window': 20,
    'label_clip_quantiles': [0.01, 0.99],
    'label_shift': -1,
    'triple_barrier_profit_target_atr_mult': 2.5,
    'triple_barrier_stop_loss_atr_mult': 1.0,
    'triple_barrier_time_horizon_bars': 12,
    'triple_barrier_atr_column': 'atr',
    'walk_forward_train_periods': 730,
    'walk_forward_test_periods': 180,
    'walk_forward_step_periods': 180,
    'walk_forward_initial_warmup': 100,
    'walk_forward_retrain_frequency_folds': 1,
    'fetch_ohlcv_limit_wfo': 5000,
    'enable_feature_selection': False,
    'feature_selection_method': 'shap',
    'num_features_to_select': 30,
    'optuna_search_spaces': {
        'n_estimators': {'type': 'int', 'low': 50, 'high': 800, 'step': 25},
        'learning_rate': {'type': 'float', 'low': 0.005, 'high': 0.1, 'log': True},
        'num_leaves': {'type': 'int', 'low': 15, 'high': 100},
        'max_depth': {'type': 'int', 'low': 2, 'high': 8},
        'lambda_l1': {'type': 'float', 'low': 1e-8, 'high': 10.0, 'log': True},
        'lambda_l2': {'type': 'float', 'low': 1e-8, 'high': 10.0, 'log': True},
        'feature_fraction': {'type': 'float', 'low': 0.4, 'high': 0.9},
        'bagging_fraction': {'type': 'float', 'low': 0.4, 'high': 0.9},
        'bagging_freq': {'type': 'int', 'low': 1, 'high': 10},
        'min_child_samples': {'type': 'int', 'low': 3, 'high': 40}
    },
    # Mapping user's 'simulation_entry/exit_order_type' to 'execution' dict
    # 'simulation_entry_order_type': 'market', # Will be handled below
    # 'simulation_exit_order_type': 'limit',   # Will be handled below
    'l2_depth_imbalance_levels': [3, 5, 10, 15, 25],
    'ta_indicator_params': {
        'rsi': {'length': 10, 'scalar': 100},
        'macd': {'fast': 8, 'slow': 21, 'signal': 5},
        'bbands': {'length': 20, 'std': 2.5}
    }
}

# Update the main 'config' dictionary with the user's exact values for the keys they provided
# and handle specific mappings.
for key, value in user_provided_config_snippet.items():
    if key == 'backtest_threshold':
        config['prediction_threshold'] = value
    elif key == 'use_shap_override':
        config['use_shap_for_importance'] = value
    # elif key == 'l2_depth_levels': # User had this, map to l2_depth_imbalance_levels for FeatureEngineer
    #     config['l2_depth_imbalance_levels'] = value
    #     # 'l2_depth' is a separate parameter for DataHandler's REST L2 snapshot
    elif key == 'simulation_entry_order_type':
        config['execution']['default_entry_order_type'] = value
    elif key == 'simulation_exit_order_type':
        config['execution']['default_exit_order_type'] = value
    else:
        config[key] = value

# Ensure base_dir is always from the environment variable
config['base_dir'] = BOT_BASE_DIR
# Correctly derive l2_collection_duration_seconds from collector_duration and collector_unit
if config.get('collector_unit') == 'minutes':
    config['l2_collection_duration_seconds'] = config.get('collector_duration', 1) * 60
elif config.get('collector_unit') == 'hours':
    config['l2_collection_duration_seconds'] = config.get('collector_duration', 1) * 3600
else: # Default to seconds if unit is unclear or missing
    config['l2_collection_duration_seconds'] = config.get('collector_duration', 60)


# --- Save Configuration to YAML File ---
config_file_path = os.path.join(BOT_BASE_DIR, 'config.yaml')
try:
    with open(config_file_path, 'w') as f:
        yaml.dump(config, f, sort_keys=False, indent=4, width=120, Dumper=yaml.SafeDumper) # Use SafeDumper
    print(f"\nConfiguration (User Updated for Phase 1) saved to: {config_file_path}")
except Exception as e:
    print(f"\nError saving configuration: {e}")
    traceback.print_exc()

# --- Display a snippet of the config for verification ---
print("\n--- Configuration Snippet (User Update for Phase 1) ---")
keys_to_display = [
    'exchange_name', 'symbol', 'timeframe', 'base_dir',
    'labeling_method', 'train_ensemble',
    'run_walk_forward_optimization', 'run_simulation_flag',
    'enable_feature_selection', 'prediction_threshold',
    'l2_depth_imbalance_levels'
]
for key in keys_to_display:
    if key in config:
        print(f"{key}: {config[key]}")
if 'optuna_search_spaces' in config and config['optuna_search_spaces']:
    print(f"optuna_search_spaces (first item key): {list(config['optuna_search_spaces'].keys())[0] if config['optuna_search_spaces'] else 'Not set'}")
if 'ta_indicator_params' in config and config['ta_indicator_params']:
     print(f"ta_indicator_params (first item key): {list(config['ta_indicator_params'].keys())[0] if config['ta_indicator_params'] else 'Not set'}")
print("-----------------------------------------")




Configuration (User Updated for Phase 1) saved to: /content/drive/MyDrive/trading_bot_project_v2/config.yaml

--- Configuration Snippet (User Update for Phase 1) ---
exchange_name: bybit
symbol: BTC/USDT
timeframe: 1m
base_dir: /content/drive/MyDrive/trading_bot_project_v2/
labeling_method: triple_barrier
train_ensemble: False
run_walk_forward_optimization: False
run_simulation_flag: True
enable_feature_selection: False
prediction_threshold: 0.2
l2_depth_imbalance_levels: [3, 5, 10, 15, 25]
optuna_search_spaces (first item key): n_estimators
ta_indicator_params (first item key): rsi
-----------------------------------------


In [3]:
# Cell 3: L2 Data Collector (Refactored Usage)

import os
import sys
import yaml
import time # For a brief pause if needed
from datetime import datetime, timezone # For logging
import traceback # For error printing

# --- Ensure BOT_BASE_DIR is available (set in Cell 1) ---
BOT_BASE_DIR = os.environ.get('BOT_BASE_DIR')
if BOT_BASE_DIR is None:
    print("CRITICAL ERROR (Cell 3): BOT_BASE_DIR is not set. Please run Cell 1 first.")
    raise EnvironmentError("BOT_BASE_DIR not set. Run Cell 1 to define it.")

# --- Add BOT_BASE_DIR to Python path to allow importing l2_data_collector ---
# This should have been done in Cell 1 already, but good to ensure for standalone cell execution if possible
if BOT_BASE_DIR not in sys.path:
    sys.path.append(BOT_BASE_DIR)
    print(f"Info (Cell 3): Added {BOT_BASE_DIR} to sys.path for L2DataCollector import.")

# --- Import the refactored class ---
L2DataCollector = None # Initialize to None
try:
    from l2_data_collector import L2DataCollector # Assumes l2_data_collector.py is in BOT_BASE_DIR
    print("L2DataCollector class imported successfully.")
except ImportError as e:
    print(f"ERROR (Cell 3): Could not import L2DataCollector: {e}")
    print(f"Please ensure 'l2_data_collector.py' is in the directory: {BOT_BASE_DIR} and that Cell 1 was run.")
except Exception as e:
    print(f"An unexpected error occurred during L2DataCollector import: {e}")
    traceback.print_exc()

# --- Load Main Configuration to Extract Collector Settings ---
# This assumes config.yaml is in BOT_BASE_DIR and was created by Cell 2.
config_file_path_main = os.path.join(BOT_BASE_DIR, 'config.yaml')
main_config_for_l2 = {} # Use a distinct variable name
collector_specific_config = {}

if os.path.exists(config_file_path_main):
    try:
        with open(config_file_path_main, 'r') as f:
            main_config_for_l2 = yaml.safe_load(f)
        if main_config_for_l2:
            print(f"Successfully loaded main configuration from {config_file_path_main} for L2 collector settings.")

            # Extract collector-specific settings from the main config.
            # The L2DataCollector class itself has defaults, but we use main_config to override them.
            # The L2DataCollector's __init__ expects a 'config' dict.
            collector_specific_config = {
                'symbol': main_config_for_l2.get('collector_symbol', 'BTCUSDT'), # L2Collector uses 'symbol' for its target
                'market_type': main_config_for_l2.get('collector_category', main_config_for_l2.get('market_type', 'linear')),
                'exchange_name': main_config_for_l2.get('exchange_name', 'bybit'),
                'l2_data_folder': main_config_for_l2.get('l2_data_folder', 'l2_data'),
                'l2_log_file': main_config_for_l2.get('l2_log_file', 'l2_data_collector.log'), # Log file name for L2 collector
                'l2_max_file_size_mb': main_config_for_l2.get('l2_max_file_size_mb', 20),
                'l2_collection_duration_seconds': main_config_for_l2.get('l2_collection_duration_seconds', 300),
                'l2_websocket_depth': main_config_for_l2.get('collector_depth', 50)
            }
            print("L2 Collector parameters extracted from main config:")
            for k, v in collector_specific_config.items():
                print(f"  {k}: {v}")
        else:
            print(f"Warning (Cell 3): Main config.yaml at {config_file_path_main} was empty. L2 Collector might use class defaults.")

    except Exception as e:
        print(f"Warning (Cell 3): Could not load or parse main config.yaml ({e}). L2 Collector might use class defaults or fail if critical params missing.")
        traceback.print_exc()
else:
    print(f"Warning (Cell 3): Main config.yaml not found at {config_file_path_main}. L2 Collector will use its internal defaults if not provided in collector_specific_config.")


# --- Instantiate and Run the Collector ---
# Check if the L2DataCollector class was successfully imported AND if we have some config for it
if L2DataCollector is not None:
    if not collector_specific_config and not main_config_for_l2: # If no config could be loaded at all
        print("ERROR (Cell 3): No configuration available for L2DataCollector. Cannot proceed.")
    else:
        print("\n--- Initializing and Starting L2 Data Collector ---")

        # The L2DataCollector's __init__ expects 'config' (which are the collector_specific_config here)
        # and 'bot_base_dir'.
        l2_collector_instance = None
        try:
            l2_collector_instance = L2DataCollector(config=collector_specific_config, bot_base_dir=BOT_BASE_DIR)

            # The start_collection_websocket method contains its own loop based on l2_collection_duration_seconds.
            # This cell will effectively block until that duration is over or an interrupt occurs.
            duration_to_run = collector_specific_config.get('l2_collection_duration_seconds', 300) # Get from extracted params
            print(f"L2 Collector is configured to run for approximately {duration_to_run / 60:.1f} minutes.")
            print("You can interrupt the kernel (Runtime -> Interrupt execution or Ctrl+M I) to stop it sooner.")
            print(f"L2 data will be saved in: {os.path.join(BOT_BASE_DIR, collector_specific_config.get('l2_data_folder', 'l2_data'))}")
            print(f"L2 collector log file: {os.path.join(BOT_BASE_DIR, collector_specific_config.get('l2_log_file', 'l2_data_collector.log'))}")

            l2_collector_instance.start_collection_websocket()

        except KeyboardInterrupt:
            print("\nL2 Data Collection interrupted by user in notebook cell.")
            if l2_collector_instance:
                l2_collector_instance.stop_collection_websocket() # Ensure graceful shutdown
        except Exception as e:
            print(f"An error occurred while running the L2 Data Collector: {e}")
            traceback.print_exc()
            if l2_collector_instance and getattr(l2_collector_instance, 'ws', None) is not None : # Check if ws object exists
                l2_collector_instance.stop_collection_websocket()
        finally:
            print("--- L2 Data Collector Cell Execution Finished ---")
            # Note: If start_collection_websocket runs in a blocking way for its duration,
            # this "finished" message will appear after the collection period.
else:
    print("\nL2DataCollector class not available (import failed). Cannot start L2 data collection.")
    print("Ensure 'l2_data_collector.py' is in your BOT_BASE_DIR and Cell 1 has been run.")

L2DataCollector class imported successfully.
Successfully loaded main configuration from /content/drive/MyDrive/trading_bot_project_v2/config.yaml for L2 collector settings.
L2 Collector parameters extracted from main config:
  symbol: BTCUSDT
  market_type: linear
  exchange_name: bybit
  l2_data_folder: l2_data
  l2_log_file: l2_data_collector.log
  l2_max_file_size_mb: 20
  l2_collection_duration_seconds: 60
  l2_websocket_depth: 50

--- Initializing and Starting L2 Data Collector ---
[2025-05-21 12:52:26 UTC] (L2Collector) L2DataCollector initialized for symbol: BTCUSDT, exchange: bybit
L2 Collector is configured to run for approximately 1.0 minutes.
You can interrupt the kernel (Runtime -> Interrupt execution or Ctrl+M I) to stop it sooner.
L2 data will be saved in: /content/drive/MyDrive/trading_bot_project_v2/l2_data
L2 collector log file: /content/drive/MyDrive/trading_bot_project_v2/l2_data_collector.log
[2025-05-21 12:52:27 UTC] (L2Collector) Using WSS URL for Bybit (linear):

In [4]:
# Cell 4: Imports & Global Setup (Main Bot)

# --- Core Libraries ---
import os
import threading
import time
import json
import traceback
import warnings
from datetime import datetime, timezone, timedelta
import sys
import yaml # For loading config
import pickle # For saving/loading ensemble models

# --- Add BOT_BASE_DIR to Python path ---
# This allows importing custom modules from this directory
# Ensure BOT_BASE_DIR is set, typically in Cell 1.
bot_base_dir = os.environ.get('BOT_BASE_DIR')
if bot_base_dir is None:
    print("CRITICAL ERROR (Cell 4): BOT_BASE_DIR is not set in environment. Please run Cell 1 first.")
    raise EnvironmentError("BOT_BASE_DIR not set. Run Cell 1 to define it.")

if BOT_BASE_DIR not in sys.path: # Should have been added in Cell 1, but double-check
    sys.path.append(BOT_BASE_DIR)
    print(f"Info (Cell 4): Added {BOT_BASE_DIR} to sys.path")

# --- Data Handling & Numerics ---
import pandas as pd
import numpy as np

# --- Machine Learning ---
import lightgbm as lgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error

# --- Load Configuration ---
print("Loading configuration...")
config_file_path = os.path.join(bot_base_dir, 'config.yaml')
config = {}
try:
    with open(config_file_path, 'r') as f:
        main_config_loader = yaml.safe_load(f)
        if main_config_loader:
            config = main_config_loader
    print(f"Configuration loaded successfully from {config_file_path}")
    if not config:
        print("Warning (Cell 4): Configuration file was empty. Using empty config dict.")
except FileNotFoundError:
    print(f"ERROR (Cell 4): Configuration file not found at {config_file_path}. Please run Cell 2 to create it.")
    print("Using empty config dict. Bot may not function correctly.")
except Exception as e:
    print(f"Error (Cell 4) loading configuration: {e}")
    traceback.print_exc()
    print("Using empty config dict. Bot may not function correctly.")

# --- Optional Libraries Setup ---
print("\nSetting up optional libraries and global flags...")
try:
    import optuna
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    HAS_OPTUNA = True
    print("Optuna loaded.")
except ImportError:
    print("WARNING (Cell 4): Optuna not found. Model training requiring it may fail.")
    HAS_OPTUNA = False
    optuna = None

try:
    from PyEMD import EMD
    HAS_PYEMD = True
    print("PyEMD (EMD-signal) library loaded successfully.")
except ImportError:
    print("WARNING (Cell 4): PyEMD (EMD-signal) not found. HHT features will be disabled.")
    HAS_PYEMD = False
    class EMD:
        def __init__(self, *args, **kwargs): pass
        def __call__(self, signal, *args, **kwargs): return np.array([])
        def get_imfs_and_residue(self): return np.array([]), np.array([])
    print("Using dummy EMD class.")

try:
    import shap
    HAS_SHAP = True
    print("SHAP library loaded successfully.")
except ImportError:
    print("WARNING (Cell 4): SHAP library not found. SHAP plots will be disabled.")
    HAS_SHAP = False
    shap = None

try:
    import matplotlib.pyplot as plt
    if config and 'plot_style' in config:
        try:
            plt.style.use(config.get('plot_style'))
            print(f"Applied plot style: {config.get('plot_style')}")
        except Exception as e_style:
            print(f"Warning (Cell 4): Could not apply plot style '{config.get('plot_style')}': {e_style}")
    HAS_MATPLOTLIB = True
    print("Matplotlib loaded.")
except ImportError:
    print("WARNING (Cell 4): Matplotlib not found. Plotting will be disabled.")
    HAS_MATPLOTLIB = False
    plt = None

try:
    from scipy.signal import hilbert
    HAS_SCIPY_HILBERT = True
    print("Scipy (signal.hilbert) loaded.")
except ImportError:
    print("WARNING (Cell 4): Scipy.signal.hilbert not found. HHT features requiring it may fail.")
    HAS_SCIPY_HILBERT = False
    def hilbert(signal, N=None, axis=-1):
        print("Error (Cell 4): hilbert function called but scipy.signal.hilbert not available.")
        return np.zeros_like(signal) + 1j * np.zeros_like(signal)
    print("Using dummy hilbert function.")

try:
    import pandas_ta as ta
    HAS_PANDAS_TA = True
    print("Pandas TA library loaded successfully.")
except ImportError:
    print("WARNING (Cell 4): pandas_ta library not found. Advanced TA features relying on it will be disabled.")
    HAS_PANDAS_TA = False
    ta = None

# --- Exchange Interaction ---
try:
    import ccxt
    print("CCXT loaded.")
except ImportError:
    print("ERROR (Cell 4): CCXT not found. Exchange interaction will fail.")
    ccxt = None

# --- Custom Helper Class Imports ---
print("\nImporting custom bot classes...")
custom_classes_to_import = [
    "AdvancedRiskManager", "SmartOrderExecutor", "DataHandler",
    "FeatureEngineer", "LabelGenerator", "ModelTrainer",
    "ModelPredictor", "StrategyBacktester", "LiveSimulator",
    "Visualizer", "TradingBotOrchestrator"
]

for class_name_str in custom_classes_to_import:
    try:
        module_name = class_name_str.lower()
        module = __import__(module_name)
        globals()[class_name_str] = getattr(module, class_name_str)
        print(f"{class_name_str} class loaded successfully from {module_name}.py")
    except ImportError as e:
        print(f"ERROR (Cell 4) importing {class_name_str} from {class_name_str.lower()}.py: {e}")
        print(f"Please ensure '{class_name_str.lower()}.py' is in '{bot_base_dir}' or Python path and contains plain Python code.")
        globals()[class_name_str] = None
    except AttributeError as e:
        print(f"ERROR (Cell 4): Attribute {class_name_str} not found in module {class_name_str.lower()}.py. Check class name. Error: {e}")
        globals()[class_name_str] = None
    except Exception as e:
        print(f"An unexpected error occurred while importing {class_name_str}: {e}")
        traceback.print_exc()
        globals()[class_name_str] = None

# --- API Key Loading (Using Colab Secrets or Environment Variables) ---
print("\nLoading API Keys...")
BYBIT_API_KEY = None
BYBIT_API_SECRET = None

# Try Colab userdata first
try:
    from google.colab import userdata
    # --- MODIFIED TO USE TESTNET KEY NAMES ---
    BYBIT_API_KEY = userdata.get("BYBIT_API_KEY_MAIN_TEST")
    BYBIT_API_SECRET = userdata.get("BYBIT_API_SECRET_MAIN_TEST")

    if BYBIT_API_KEY and BYBIT_API_SECRET:
        print("TESTNET API Keys loaded successfully from Colab secrets.")
    else:
        print("*** WARNING: TESTNET API Keys not found or empty in Colab secrets ('BYBIT_API_KEY_MAIN_TEST', 'BYBIT_API_SECRET_MAIN_TEST'). ***")
        BYBIT_API_KEY = None
        BYBIT_API_SECRET = None
except ImportError:
    print("Not in Colab environment. Will check environment variables for API keys.")
    # If not found in Colab secrets, try environment variables
    if not (BYBIT_API_KEY and BYBIT_API_SECRET): # Check again in case Colab import failed but env vars might exist
        BYBIT_API_KEY = os.environ.get("BYBIT_API_KEY_MAIN_TEST") # Check for TESTNET env vars
        BYBIT_API_SECRET = os.environ.get("BYBIT_API_SECRET_MAIN_TEST")
        if BYBIT_API_KEY and BYBIT_API_SECRET:
            print("TESTNET API Keys loaded successfully from environment variables.")
        else:
            print("WARNING (Cell 4): TESTNET API Keys not found in Colab secrets or environment variables ('BYBIT_API_KEY_MAIN_TEST', 'BYBIT_API_SECRET_MAIN_TEST'). Live trading requiring authentication will fail if testnet keys are intended.")
except Exception as e: # Catch other potential errors during userdata.get()
    print(f"An unexpected error occurred loading secrets: {e}")
    traceback.print_exc()
    BYBIT_API_KEY = None
    BYBIT_API_SECRET = None

# --- Warnings Configuration ---
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

print("\nImports and global setup complete.")
if config:
    print(f"Config dictionary loaded with {len(config)} top-level keys.")
    print(f"Config 'exchange_testnet' is set to: {config.get('exchange_testnet')}")
else:
    print("CRITICAL WARNING (Cell 4): Config dictionary is empty or failed to load. Bot will likely not function correctly.")


Loading configuration...
Configuration loaded successfully from /content/drive/MyDrive/trading_bot_project_v2/config.yaml

Setting up optional libraries and global flags...
Optuna loaded.
PyEMD (EMD-signal) library loaded successfully.
SHAP library loaded successfully.
Applied plot style: seaborn-v0_8-darkgrid
Matplotlib loaded.
Scipy (signal.hilbert) loaded.
Pandas TA library loaded successfully.
CCXT loaded.

Importing custom bot classes...
AdvancedRiskManager class loaded successfully from advancedriskmanager.py
SmartOrderExecutor class loaded successfully from smartorderexecutor.py
DataHandler class loaded successfully from datahandler.py
FeatureEngineer class loaded successfully from featureengineer.py
LabelGenerator class loaded successfully from labelgenerator.py
ModelTrainer class loaded successfully from modeltrainer.py
ModelPredictor class loaded successfully from modelpredictor.py
StrategyBacktester class loaded successfully from strategybacktester.py
LiveSimulator class loa

In [5]:
# ================================================================
# Cell 5 · Trading Bot Orchestrator Initialization   (patched)
# ================================================================
import os, traceback, inspect

# ---------- 0 · Pre-flight checks --------------------------------
if "BOT_BASE_DIR" not in os.environ:
    raise EnvironmentError("Cell 5: BOT_BASE_DIR not set – run Cell 1 first.")
BOT_BASE_DIR = os.environ["BOT_BASE_DIR"]
print(f"Info:  BOT_BASE_DIR → {BOT_BASE_DIR}")

if "config" not in globals() or not isinstance(config, dict) or not config:
    raise ValueError("Cell 5: global 'config' missing or empty – did you run Cells 2 & 4?")
print(f"Info:  base config has {len(config)} keys.")

# ---------- 1 · Build Bybit-Testnet exchange_kwargs --------------
exchange_kwargs = {
    "enableRateLimit": True,
    "options": {
        "defaultType": "linear",          # USDT-perps
        "fetchCurrencies": False,         # skip private /asset/ call
        "urls": {                         # point both routes at test-net
            "api": {
                "public":  "https://api-testnet.bybit.com",
                "private": "https://api-testnet.bybit.com",
            }
        },
    },
    # no apiKey / secret → public-only access
}

# ---------- 2 · Merge into notebook-level config -----------------
config.update({
    "exchange_name":        "bybit",
    "exchange_kwargs":      exchange_kwargs,
    "allow_no_exchange_init": False,      # fail hard if even public init breaks
})

# ---------- 3 · Collect global library flags / modules -----------
global_library_flags = {
    name: globals().get(name, False) for name in [
        "HAS_OPTUNA", "HAS_PYEMD", "HAS_SCIPY_HILBERT",
        "HAS_SHAP", "HAS_MATPLOTLIB", "HAS_PANDAS_TA"
    ]
}
global_library_modules = {
    name: globals().get(name) for name in [
        "optuna", "EMD", "hilbert", "shap", "plt", "ta", "ccxt"
    ]
}

# ---------- 4 · Instantiate TradingBotOrchestrator ---------------
print("\n--- Initialising TradingBotOrchestrator ---")
try:
    sig = inspect.signature(TradingBotOrchestrator)
    if "bot_base_dir" in sig.parameters:                   # old signature
        bot_orchestrator = TradingBotOrchestrator(
            config=config,
            bot_base_dir=BOT_BASE_DIR,
            global_library_flags=global_library_flags,
            global_library_modules=global_library_modules,
            api_key=None, api_secret=None,                # public-only
        )
    else:                                                  # new signature – dir in config
        config["bot_base_dir"] = BOT_BASE_DIR
        bot_orchestrator = TradingBotOrchestrator(
            config=config,
            global_library_flags=global_library_flags,
            global_library_modules=global_library_modules,
            api_key=None, api_secret=None,
        )

    if bot_orchestrator.exchange:
        print(f"✅  Orchestrator ready – {len(bot_orchestrator.exchange.symbols)} symbols loaded from Bybit test-net")
    else:
        print("⚠️  Orchestrator initialised but exchange is None (check allow_no_exchange_init flag)")

except Exception as e:
    print(f"❌  Failed to initialise TradingBotOrchestrator: {e}")
    traceback.print_exc()
    bot_orchestrator = None

print("\n--- Cell 5 done ---")

Info:  BOT_BASE_DIR → /content/drive/MyDrive/trading_bot_project_v2/
Info:  base config has 97 keys.

--- Initialising TradingBotOrchestrator ---
Exchange 'bybit' initialized in SANDBOX/TESTNET mode.
Markets loaded for 'bybit'.
AdvancedRiskManager initialized.
SmartOrderExecutor initialized.
DataHandler initialized. OHLCV path: /content/drive/MyDrive/trading_bot_project_v2/ohlcv_data_BTC_USDT_1m.csv, L2 Raw path: /content/drive/MyDrive/trading_bot_project_v2/l2_data/btcusdt_l2_data_1m.jsonl
FeatureEngineer initialized (Phase 1 Update).
LabelGenerator initialized (Phase 1 Update). Using method: triple_barrier
ModelTrainer initialized (Phase 1 Update).
ModelPredictor initialized.
StrategyBacktester initialized.
LiveSimulator initialized.
Visualizer initialized.
TradingBotOrchestrator initialized (Phase 1 Update).
✅  Orchestrator ready – 2421 symbols loaded from Bybit test-net

--- Cell 5 done ---


In [6]:
# Cell 5: Trading Bot Orchestrator Initialization

# This cell instantiates the main TradingBotOrchestrator class.
# The complex logic of the old CombinedTradingBot has been moved to specialized classes
# which were imported in Cell 4 and are instantiated by the TradingBotOrchestrator.

import os # For BOT_BASE_DIR check
import traceback # For printing stack trace on error

# --- Ensure prerequisite variables from Cell 1 and Cell 4 are available ---
# These checks are important for notebook execution flow.
if 'BOT_BASE_DIR' not in os.environ or os.environ.get('BOT_BASE_DIR') is None:
    print("CRITICAL ERROR (Cell 5): BOT_BASE_DIR is not set. Please run Cell 1 first.")
    raise EnvironmentError("BOT_BASE_DIR not set. Ensure Cell 1 has been executed.")
else:
    bot_base_dir_check = os.environ.get('BOT_BASE_DIR') # To use in this cell if needed, already global from Cell 4
    print(f"Info (Cell 5): Using BOT_BASE_DIR: {bot_base_dir_check}")


if 'config' not in globals() or not isinstance(config, dict) or not config:
    print("CRITICAL ERROR (Cell 5): 'config' dictionary not found, not a dict, or is empty.")
    print("Please ensure Cell 2 (Config Settings) and Cell 4 (Imports & Global Setup) have been run successfully.")
    raise ValueError("'config' dictionary not available or invalid. Ensure Cell 2 and Cell 4 have been executed.")
else:
    print(f"Info (Cell 5): 'config' dictionary loaded with {len(config)} keys.")

print("\n--- Initializing Trading Bot Orchestrator ---")

# Gather global library flags and modules (expected to be set in Cell 4)
# These are passed to the orchestrator to inform component initializations.
# Using globals().get() for safety, providing False/None as defaults.
global_library_flags = {
    'HAS_OPTUNA': globals().get('HAS_OPTUNA', False),
    'HAS_PYEMD': globals().get('HAS_PYEMD', False),
    'HAS_SCIPY_HILBERT': globals().get('HAS_SCIPY_HILBERT', False),
    'HAS_SHAP': globals().get('HAS_SHAP', False),
    'HAS_MATPLOTLIB': globals().get('HAS_MATPLOTLIB', False),
    'HAS_PANDAS_TA': globals().get('HAS_PANDAS_TA', False)
}
print(f"Info (Cell 5): Global library flags collected: {global_library_flags}")

global_library_modules = {
    'optuna': globals().get('optuna'),
    'EMD': globals().get('EMD'),
    'hilbert': globals().get('hilbert'),
    'shap': globals().get('shap'),
    'plt': globals().get('plt'),
    'ta': globals().get('ta'),
    'ccxt': globals().get('ccxt') # Pass the ccxt module itself
}
# Simple check for modules
# for mod_name, mod_obj in global_library_modules.items():
#     print(f"Info (Cell 5): Module '{mod_name}' is {'available' if mod_obj else 'NOT available (or dummy)'}")


# Check if TradingBotOrchestrator class was imported successfully in Cell 4
bot_orchestrator = None # Initialize to None
if 'TradingBotOrchestrator' in globals() and TradingBotOrchestrator is not None:
    try:
        print("Instantiating TradingBotOrchestrator...")
        bot_orchestrator = TradingBotOrchestrator(
            config=config,
            api_key=globals().get('BYBIT_API_KEY'),
            api_secret=globals().get('BYBIT_API_SECRET'),
            global_library_flags=global_library_flags,
            global_library_modules=global_library_modules
        )

        # Check if the exchange component within the orchestrator was initialized
        if bot_orchestrator.exchange:
            print("TradingBotOrchestrator instance created and exchange initialized successfully.")
        elif config.get('allow_no_exchange_init', False): # If no exchange is okay (e.g. offline analysis)
            print("Info (Cell 5): TradingBotOrchestrator instance created, but without a live exchange connection (as per 'allow_no_exchange_init' config). Some functionalities like live trading/simulation will be unavailable.")
        else:
            # This case means exchange init failed AND it was required
            print("CRITICAL (Cell 5): TradingBotOrchestrator instance created, but its exchange component FAILED to initialize, and 'allow_no_exchange_init' is False. Live trading/simulation and data fetching will not work.")
            # Depending on the desired behavior, you might want to nullify bot_orchestrator here or raise an error
            # to prevent Cell 6 from running with a non-functional orchestrator.
            # For now, it will proceed but Cell 6 should check bot_orchestrator.exchange.

    except Exception as e:
        print(f"ERROR (Cell 5): Failed to instantiate TradingBotOrchestrator: {e}")
        traceback.print_exc()
        bot_orchestrator = None # Ensure it's None on failure
else:
    print("ERROR (Cell 5): TradingBotOrchestrator class not imported or not available.")
    print("Please ensure Cell 4 ran successfully and 'trading_bot_orchestrator.py' is correct and in the BOT_BASE_DIR.")

# The old CombinedTradingBot class definition should be REMOVED from this cell.
# All its logic is now distributed across the imported .py files and managed by TradingBotOrchestrator.

print("\n--- Orchestrator Initialization Attempt Finished ---")
if bot_orchestrator:
    print(f"Orchestrator object: {bot_orchestrator}")
    if bot_orchestrator.exchange:
        print(f"Orchestrator exchange object: {bot_orchestrator.exchange.id if bot_orchestrator.exchange else 'None'}")
else:
    print("Orchestrator object is None (failed to initialize).")

Info (Cell 5): Using BOT_BASE_DIR: /content/drive/MyDrive/trading_bot_project_v2/
Info (Cell 5): 'config' dictionary loaded with 99 keys.

--- Initializing Trading Bot Orchestrator ---
Info (Cell 5): Global library flags collected: {'HAS_OPTUNA': True, 'HAS_PYEMD': True, 'HAS_SCIPY_HILBERT': True, 'HAS_SHAP': True, 'HAS_MATPLOTLIB': True, 'HAS_PANDAS_TA': True}
Instantiating TradingBotOrchestrator...
Exchange 'bybit' initialized in SANDBOX/TESTNET mode.
Markets loaded for 'bybit'.
AdvancedRiskManager initialized.
SmartOrderExecutor initialized.
DataHandler initialized. OHLCV path: /content/drive/MyDrive/trading_bot_project_v2/ohlcv_data_BTC_USDT_1m.csv, L2 Raw path: /content/drive/MyDrive/trading_bot_project_v2/l2_data/btcusdt_l2_data_1m.jsonl
FeatureEngineer initialized (Phase 1 Update).
LabelGenerator initialized (Phase 1 Update). Using method: triple_barrier
ModelTrainer initialized (Phase 1 Update).
ModelPredictor initialized.
StrategyBacktester initialized.
LiveSimulator initializ

In [7]:
# Cell 6: Main Execution Workflow

# This cell assumes that 'bot_orchestrator' has been successfully initialized in Cell 5,
# and 'config' (the configuration dictionary) is also available from Cell 4.
# It also assumes that all necessary library flags (e.g., HAS_MATPLOTLIB) and
# modules (e.g., plt) are globally available from Cell 4.

import pandas as pd # For pd.option_context
import traceback # For printing stack trace on error
import sys # For checking interactive mode

if 'bot_orchestrator' in globals() and bot_orchestrator is not None and \
   (bot_orchestrator.exchange or config.get('allow_no_exchange_init', False)):
    print("\n--- Starting Main Execution Workflow via Orchestrator ---")

    # --- CHOOSE YOUR WORKFLOW ---
    # Set 'RUN_WALK_FORWARD' to True to execute Walk-Forward Optimization.
    # Otherwise, the standard single train/backtest/simulation workflow will run.
    RUN_WALK_FORWARD = config.get('run_walk_forward_optimization', False) # Get from config

    if RUN_WALK_FORWARD:
        print("\n*** EXECUTING WALK-FORWARD OPTIMIZATION WORKFLOW ***")
        try:
            bot_orchestrator.run_full_workflow(run_wfo=True)
        except Exception as e:
            print(f"An error occurred during the Walk-Forward Optimization workflow: {e}")
            traceback.print_exc()
    else:
        print("\n*** EXECUTING STANDARD WORKFLOW (Single Train/Backtest/Sim) ***")
        try:
            bot_orchestrator.run_full_workflow(run_wfo=False)
        except Exception as e:
            print(f"An error occurred during the standard workflow: {e}")
            traceback.print_exc()

    # --- Granular Workflow Control (Optional - Uncomment sections to run specific parts) ---
    # This is useful for debugging or re-running specific stages after making config changes.
    # Ensure that the necessary preceding stages have been successfully completed or data is available.

    # print("\n--- Granular Workflow Control (Example) ---")

    # --- Stage 1: Prepare Data for Training ---
    # data_prepared_successfully = False
    # print("\n--- Stage 1: Preparing Data ---")
    # try:
    #     # For a standard run, df_input can be None to load full history.
    #     # For re-running a specific WFO fold's data prep, you'd need to load that slice.
    #     if bot_orchestrator.prepare_data_for_training(df_input=None, save_features=True, save_ohlcv=True):
    #         print("Data preparation for training was successful.")
    #         data_prepared_successfully = True
    #         # You can inspect the prepared dataframes:
    #         # print("Historical Data Head:\n", bot_orchestrator.df_historical_data.head())
    #         # print("Features DataFrame Head:\n", bot_orchestrator.df_features.head())
    #         # print("Labeled Features DataFrame Head:\n", bot_orchestrator.df_labeled_features.head())
    #     else:
    #         print("Data preparation for training failed. Check logs.")
    # except Exception as e:
    #     print(f"Error during data preparation: {e}")
    #     traceback.print_exc()

    # --- Stage 2: Train Model ---
    # model_trained_successfully = False
    # if data_prepared_successfully:
    #     print("\n--- Stage 2: Training Model ---")
    #     try:
    #         # Trains on self.df_labeled_features by default if df_training_data is None
    #         if bot_orchestrator.train_model(save_model=True):
    #             print("Model training was successful.")
    #             model_trained_successfully = True
    #             # print("Trained features list:", bot_orchestrator.trained_features_list)
    #         else:
    #             print("Model training failed. Check logs.")
    #     except Exception as e:
    #         print(f"Error during model training: {e}")
    #         traceback.print_exc()
    # else:
    #     print("Skipping model training (data not prepared).")

    # --- Stage 3: Run Backtest ---
    # backtest_run_successfully = False
    # # A model needs to be available either from current session training or loaded.
    # model_is_ready_for_backtest = (bot_orchestrator.trained_model_booster is not None or
    #                                bot_orchestrator.trained_ensemble_models is not None or
    #                                (bot_orchestrator.model_predictor and bot_orchestrator.model_predictor.model_object is not None))
    # if model_is_ready_for_backtest or config.get('allow_backtest_load_model_directly', True): # Add flag to allow direct load
    #     print("\n--- Stage 3: Running Backtest ---")
    #     try:
    #         # df_backtest_data=None uses self.df_features. load_latest_model=True reloads from disk.
    #         # If model was just trained, set load_latest_model=False to use in-memory model.
    #         backtest_results_df, trades_log_df = bot_orchestrator.run_backtest(df_backtest_data=None, load_latest_model=not model_trained_successfully)
    #         if backtest_results_df is not None and not backtest_results_df.empty:
    #             print("Backtest finished.")
    #             if trades_log_df is not None and not trades_log_df.empty:
    #                 print("\nBacktest Trades Log Sample:")
    #                 with pd.option_context('display.max_rows', 10, 'display.max_columns', None, 'display.width', 1000):
    #                     print(trades_log_df.head())
    #             elif trades_log_df is not None:
    #                 print("\nNo trades executed during backtest.")
    #             backtest_run_successfully = True
    #         else:
    #             print("Backtest failed to produce results. Check logs.")
    #     except Exception as e:
    #         print(f"Error during backtest: {e}")
    #         traceback.print_exc()
    # else:
    #     print("Skipping backtest (model not trained or loaded).")

    # --- Stage 4: Run Live Simulation (Optional) ---
    # model_is_ready_for_simulation = (bot_orchestrator.trained_model_booster is not None or
    #                                  bot_orchestrator.trained_ensemble_models is not None or
    #                                  (bot_orchestrator.model_predictor and bot_orchestrator.model_predictor.model_object is not None))
    # if model_is_ready_for_simulation or config.get('allow_simulation_load_model_directly', True):
    #     if config.get('run_simulation_flag', False): # Check the master flag
    #         print("\n--- Stage 4: Running Live Simulation ---")
    #         try:
    #             if bot_orchestrator.live_simulator:
    #                 bot_orchestrator.run_live_simulation()
    #             else:
    #                 print("Live simulator component not available. Skipping simulation.")
    #         except Exception as e:
    #             print(f"Error during live simulation: {e}")
    #             traceback.print_exc()
    #             if bot_orchestrator.live_simulator and bot_orchestrator.live_simulator.simulation_running:
    #                 bot_orchestrator.live_simulator.stop_live_simulation()
    #     else:
    #         print("\nSkipping Live Simulation (run_simulation_flag is False in config).")
    # else:
    #     print("Skipping live simulation (model not trained or loaded).")

    # --- Stage 5: Visualize Other Results (Feature Importance, EMD, etc.) ---
    # print("\n--- Stage 5: Visualizing Other Results ---")
    # try:
    #     if bot_orchestrator.visualizer:
    #         bot_orchestrator.visualize_results()
    #     else:
    #         print("Visualizer not available, skipping additional visualizations.")
    # except Exception as e:
    #     print(f"Error during visualization: {e}")
    #     traceback.print_exc()

    print("\n--- Orchestrator Workflow Attempt Finished ---")
else:
    print("\nSkipping Main Execution Workflow: Orchestrator not initialized, exchange failed, or 'allow_no_exchange_init' is False with no exchange.")
    print("Please ensure Cell 1 (Setup), Cell 2 (Config), Cell 4 (Imports), and Cell 5 (Orchestrator Init) have run successfully.")

print("\n="*50)
print("Notebook Execution Finished")
print("="*50)

# Optional: Keep plots open if not in interactive mode (e.g., running as script)
# This part is from your original notebook.
# Ensure 'plt' and 'HAS_MATPLOTLIB' are accessible here (defined in Cell 4)
# if 'plt' in globals() and plt is not None and \
#    'HAS_MATPLOTLIB' in globals() and HAS_MATPLOTLIB and \
#    'sys' in globals() : # Check if sys was imported
#      INTERACTIVE_MODE = 'ipykernel' in sys.modules
#      if not INTERACTIVE_MODE:
#          print("Displaying plots. Close plot windows to exit if any were generated and `show_plots` is True in config.")
#          plt.show() # This will show all figures generated if plt.show() wasn't called in Visualizer


--- Starting Main Execution Workflow via Orchestrator ---

*** EXECUTING STANDARD WORKFLOW (Single Train/Backtest/Sim) ***

--- Starting Data Preparation ---
No input DataFrame provided, loading full historical data.
Loading existing OHLCV data from /content/drive/MyDrive/trading_bot_project_v2/ohlcv_data_BTC_USDT_1m.csv
Loaded 999 records from CSV.
Cleaned OHLCV data. Shape: (999, 6)
Loading and aligning historical L2 data...
Loaded 2621 L2 records from /content/drive/MyDrive/trading_bot_project_v2/l2_data/btcusdt_l2_data_1m.jsonl
Warning (DataHandler): 999 OHLCV candles could not be aligned with preceding L2 data.
Aligned L2 data with OHLCV. Resulting shape: (999, 8)
Data loading and preparation complete. Final DataFrame shape: (999, 8)
Starting feature generation for 999 rows...
Warning (FeatureEngineer): Error calculating TA feature 'macd': Cannot set a DataFrame with multiple columns to the single column macd
Warning (FeatureEngineer): Error calculating TA feature 'bbands': Canno